In [ ]:
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import shutil

print("Загрузка полного датасета XLSum English...")
train_url = "https://huggingface.co/api/datasets/csebuetnlp/xlsum/parquet/english/train/0.parquet"
test_url = "https://huggingface.co/api/datasets/csebuetnlp/xlsum/parquet/english/test/0.parquet"

df_train_full = pd.read_parquet(train_url)
df_test_full = pd.read_parquet(test_url)

tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

# Фильтрация записей. Оставляем только те, которые короче 512 токенов
def is_short_enough(text):
    # Быстрая проверка: если слов меньше 400, скорее всего влезет (экономим время на токенизации)
    if len(str(text).split()) <= 400: return True
    # Точная проверка для пограничных случаев
    return len(tokenizer.encode(str(text))) <= 512

print("Фильтрация данных ...")
df_train_filtered = df_train_full[df_train_full['text'].apply(is_short_enough)]
df_test_filtered = df_test_full[df_test_full['text'].apply(is_short_enough)]

train_data = Dataset.from_pandas(df_train_filtered)
test_data = Dataset.from_pandas(df_test_filtered)

print(f"Итого для обучения: {len(train_data)} примеров (отфильтровано {(len(df_train_full)-len(df_train_filtered))} длинных статей)")

# Токенизация датасета
def preprocess_function(examples):
    inputs = ["summarize: " + str(doc) for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("Токенизация отфильтрованных данных...")
tokenized_train = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
tokenized_test = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)

# Модель и обучение
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  
    eval_steps=500,             
    learning_rate=3e-5,           
    per_device_train_batch_size=8, 
    num_train_epochs=1,           
    weight_decay=0.01,
    save_total_limit=2,           
    logging_steps=100,
    fp16=True,              
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

print("Запуск масштабного обучения...")
trainer.train()

# Сохранение
save_path = "./my_full_model_t5"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
shutil.make_archive("full_model_weights", 'zip', save_path)

print("Модель готова! 'full_model_weights.zip'")